## QUESTIONS:

PART 1: 
Write a Python function that takes the name of a star and returns a galpy Orbit object (setup with the star's position, distance, proper motion, and radial velocity). Apply to: GJ 440, Lacaille 8760, Vega. Check that you are finding the right star.

PART 2: 
Match the APOGEE-red-clump catalog to Gaia DR2, integrate the orbits using galpy for 20 Gyr, and make a density plot in Galactocentric X and Y coordinates of all of the positions of the stars over the next 20 Gyr.

PART 3: 
Write a function that takes (ra,dec,distance,proper motion ra,proper motion dec,line-of-sight velocity) and their Gaussian uncertainties for a star and computes eccentricity, zmax, rperi, rap and their uncertainties using Monte Carlo sampling with the galpy function galpy.actionAngle.actionAngleStaeckel.EccZmaxRperiRap. Apply to some stars in the Gaia RV sample. 


## SOLUTIONS:

PART 1:

Input name of star. We want to test 3 stars: GJ 440, Lacaille 8760, and Vega. Of course, our function will be generalized to any star identifiable by SIMBAD (Set of Identifications, Measurements, and Bibliography for Astronomical Data)

In [ ]:
# starname = input("Input star name to return as galpy Orbit object")
# Testing Examples:
starnames = ["GJ 440", "Lacaille 8760", "Vega"]

In [ ]:
# importing necessary modules in the order used
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.gaia import Gaia
from galpy.orbit import Orbit

Algorithm:

1) Setup SIMBAD search to also accept: parallax, proper motion (right ascention direction), proper motion (declination direction) and radial velocity using add_votable_fields (VOTable - Virtual Observatory Table)

2) Given name of star, identify star in SIMBAD database using the query_object function and retrieve info.


3) Since SIMBAD returns RA & DEC in hours/mins/seconds & degrees/arcminutes/arcseconds respectively, we use Skycoord to convert both into float degrees (ICRS (International Celestial Reference System) units) as that is what a galpy Orbit object requires. This makes a Skycoord object.


4) Combine simbad_converted with simbad_info to make simbad_query ready to be queried by ADQL (Astronomical Data Query Language). 6 variables are required for  epoch propagation.


5) In case SIMBAD returns "--" for unknown value, set values to 0.


6) Perform cone search in GAIA DR2 with 0.000001 degrees and using epoch adjustment in ADQL by EPOCH_PROP_POS (we are propagating from 2000 to 2015.5).


7) If Gaia tables are not empty, at least 5 tables (maybe not radial velocity) should have values. If not, use SIMBAD data to make a galpy Orbit object. Either way, if radial velocity data is lacking, set it = 20 km/s.


In [ ]:
def starname_to_galpyorbitobject(starname):
    
    simbad_setup = Simbad()  # Step 1
    simbad_setup.add_votable_fields("parallax", "pmra", "pmdec", "rv_value")
    
    simbad_info = simbad_setup.query_object(starname)  # Step 2   
    
    
    simbad_converted = SkyCoord(simbad_info['RA'][0], simbad_info['DEC'][0], # Step 3
                                unit = (u.hourangle, u.deg))  
    
    simbad_query = [simbad_converted.ra.value,  # Step 4
                    simbad_converted.dec.value,
                    simbad_info['PLX_VALUE'][0],
                    simbad_info['PMRA'][0],
                    simbad_info['PMDEC'][0],
                    simbad_info['RV_VALUE'][0]]
    
    
    simbad_query_fixed = []  # Step 5
    for value in simbad_query:
        if str(value) != '--':
            simbad_query_fixed.append(value)
        else:
            simbad_query_fixed.append(0.0)
        
    if simbad_query_fixed[5] == 0.0:
        simbad_query_fixed[5] = 20.0
            
    radius = 0.000001*u.deg  # Step 6
    query =  """
             SELECT source_id, ra, dec, pmra, pmdec, parallax, radial_velocity
             FROM gaiadr2.gaia_source
             WHERE 1=CONTAINS(
             POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),
             CIRCLE('ICRS', 
             COORD1(EPOCH_PROP_POS({0},{1},{2},{3},{4},{5},2000,2015.5)),
             COORD2(EPOCH_PROP_POS({0},{1},{2},{3},{4},{5},2000,2015.5)), {6}))
             """.format(*simbad_query_fixed, radius.value)
    gaiadr2_info = Gaia.launch_job_async(query)
    results = gaiadr2_info.get_results()
    
    if len(results) == 0:  # Step 7
        simbad_query_fixed[2] = 1 / simbad_query_fixed[2]  # parallax to distance
        orbit = Orbit(vxvv = (simbad_query_fixed), radec = True)
        return orbit
    else:
        if results['radial_velocity'][0] == '--':
            results['radial_velocity'][0] = 20
        orbit = Orbit(vxvv = (results['ra'][0],
                              results['dec'][0],
                              1 / results['parallax'][0],
                              results['pmra'][0],
                              results['pmdec'][0],
                              results['radial_velocity'][0]), radec = True)
        return orbit

In [ ]:
# testing out inputted stars
for star in starnames:
    orbit = starname_to_galpyorbitobject(star)
    print(type(orbit))
    print("Star:", star)
    print("RA:", orbit.ra(), "deg")
    print("DEC:", orbit.dec(), "deg")
    print("Distance:", orbit.dist(), "kpc")
    print("Proper Motion RA:", orbit.pmra(), "mas/yr")
    print("Proper Motion DEC:", orbit.pmdec(), "deg")
    print("Radial Velocity:", orbit.vlos(), "km/s\n")

PART 2:

In [ ]:
# importing necessary modules in the order used, 
# assume modules in PART 1 are still being used
import gaia_tools.load as gload
import numpy as np
from galpy.potential import MWPotential2014
from scipy.stats import kde


import matplotlib.pyplot as plt


In [ ]:
# Since I had trouble running my python function to download data online, 
# I manually downloaded APOGEE-RC DR14 and the crossmatched GAIA DR2 stars
# So, this function calls files locally and does not require internet.

rc_cat, gaia2_matches= gload.apogeerc(xmatch='gaiadr2')
# I made changes to gaia_tools/xmatch/__init__.py source file, 
# I changed 'float128' to 'float64' in line64 as I kept getting a float128 error

In [ ]:
# This quickly checks how many stars are crossmatched.
counter = 0
for i in gaia2_matches:
    counter +=1
print (counter)

In [ ]:
ts = np.linspace(0, 20, 10)*u.Gyr  # to integrate orbits over 20 Billion years
x = []  # store all x points
y = []  # story all y points
counter = 0
for star in range(np.size(gaia2_matches)):
    # stars with no radial velocity recorded are removed 
    # by default, they are -9999.99 km/s which is obviously incorrect
    if gaia2_matches[star]["radial_velocity"] !=  -9999.99:
        o = Orbit(vxvv=[gaia2_matches[star]["ra_epoch2000"],
                        gaia2_matches[star]["dec_epoch2000"],
                        (1/gaia2_matches[star]["parallax"]),
                        gaia2_matches[star]["pmra"],
                        gaia2_matches[star]["pmdec"],
                        gaia2_matches[star]["radial_velocity"]], 
                        radec = True, ro = 8., vo = 220.)
        o.integrate(ts, MWPotential2014)
        info = o.plot(d1='x',d2='y')
        x_temp, y_temp = info[0].get_data()
        x.extend(x_temp)
        y.extend(y_temp)
        plt.close()
        print(counter)
        counter +=1
        
        



In [ ]:
# Make density plot (2D histogram)
plt.hist2d(x, y, bins=(100, 100), cmap=plt.cm.jet)



plt.colorbar()
plt.savefig('density_plot.png', bbox_inches='tight')
plt.savefig('density_plot.pdf', bbox_inches='tight')